<a href="https://colab.research.google.com/github/rafaeldsouza/mestrado-ReconhecimentoPadrao/blob/main/MCAP_Reconhecimento_Padroes_Atividade4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Aluno:** Rafael de Souza Conceição

**Matéria:**Reconhecimento de Padrões - Francisco de Assis Boldt


Escolha uma base de classificação e compare os classificadores Logistic Regression e KNN do scikit-learn.

Use pelo menos duas formas de avaliação e as repita pelo menos 10 vezes.

Calcule a média das repetições de cada avaliação.

#### Baixar dataset do kaggle

In [1]:
!pip install kaggle

**Enviar as credencias do Kaggle**

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [4]:
!kaggle datasets download "ronitf/heart-disease-uci"

  0% 0.00/3.40k [00:00<?, ?B/s]
100% 3.40k/3.40k [00:00<00:00, 5.97MB/s]


In [5]:
!unzip heart-disease-uci.zip

Archive:  heart-disease-uci.zip
  inflating: heart.csv               


In [93]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


In [69]:
x = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dfNormal = pd.DataFrame(x_scaled)
dfNormal.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.708333,1.0,1.000000,0.481132,0.244292,1.0,0.0,0.603053,0.0,0.370968,0.0,0.0,0.333333,1.0
1,0.166667,1.0,0.666667,0.339623,0.283105,0.0,0.5,0.885496,0.0,0.564516,0.0,0.0,0.666667,1.0
2,0.250000,0.0,0.333333,0.339623,0.178082,0.0,0.0,0.770992,0.0,0.225806,1.0,0.0,0.666667,1.0
3,0.562500,1.0,0.333333,0.245283,0.251142,0.0,0.5,0.816794,0.0,0.129032,1.0,0.0,0.666667,1.0
4,0.583333,0.0,0.000000,0.245283,0.520548,0.0,0.5,0.702290,1.0,0.096774,1.0,0.0,0.666667,1.0


In [80]:
classificadores = [KNeighborsClassifier(7),LogisticRegression()]
repetir = 10

In [95]:
def treino(X_train, X_test, y_train, y_test):
  resultadosCalculado = []
  nomeClassificador=[]
  for i in range(len(classificadores)): 
    classificador = classificadores[i]
    resultados = []
    for j in range(repetir):
      inicio = time.time()
      classificador.fit(X_train, y_train.ravel())
      fim = time.time()
      tempoTreino = fim - inicio
      
      inicio = time.time()
      y_inference = classificador.predict(X_test)
      fim = time.time()
      tempoInferencia = fim - inicio
      
      acuracia = accuracy_score(y_test, y_inference)
      precision = precision_score(y_test, y_inference)
      recall = recall_score(y_test, y_inference)
    
      resultados.append([acuracia,precision,recall,tempoTreino,tempoInferencia])
    resultadosCalculado.append(np.average(resultados, axis=0))
    nomeClassificador.append(type(classificador).__name__)

  dfResultado = pd.DataFrame(data=resultadosCalculado, index=nomeClassificador, 
                                  columns=["Média Acuracia","Média Precisao","Média Recall","Média Tempo_treino","Média Tempo_inferencia"])
  return dfResultado 
  

##Dados normalizados

In [96]:
feature = dfNormal.iloc[:,0:13] 
target = dfNormal.iloc[:,13]

In [97]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=42)

In [98]:
treino(X_train, X_test, y_train, y_test)

,Média Acuracia,Média Precisao,Média Recall,Média Tempo_treino,Média Tempo_inferencia
KNeighborsClassifier,0.769231,0.837209,0.72,0.001386,0.004204
LogisticRegression,0.791209,0.829787,0.78,0.004947,0.000575


##Dados não normalizados

In [99]:
feature = df.iloc[:,0:13] 
target = df.iloc[:,13]

In [100]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.3, random_state=42)

In [101]:
treino(X_train, X_test, y_train, y_test)

,Média Acuracia,Média Precisao,Média Recall,Média Tempo_treino,Média Tempo_inferencia
KNeighborsClassifier,0.670330,0.685185,0.74,0.001605,0.004667
LogisticRegression,0.813187,0.823529,0.84,0.020857,0.000806
